# Wrangle Data
#### Danny Fay

In [1]:
# Import necessary packages
import os
import numpy as np
import pandas as pd

sc

In [2]:
# Project file path
bus_data_folder = 'data/external/bus_time'

In [3]:
# Define extract buses mapping function and find direction
def extractBuses(partId, list_of_records):
    if partId==0:
        list_of_records.next()
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row[7].split('_')) >= 3:
            bus = row[7].split('_')[2][:3]
            if bus in bus_list:
                direction = 0
            
                if bus == 'B15':
                    if 50 <= float(row[4]) < 250:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B4':
                    if 100 <= float(row[4]) < 250:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B3':
                    if 125 <= float(row[4]) < 250:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B8':
                    if 150 <= float(row[4]) < 300:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B38':
                    if 100 <= float(row[4]) < 250:
                        direction = 1
                    else:
                        direction = 2   

                elif bus == 'B43':
                    if 140 <= float(row[4]) < 320:
                        direction = 1
                    else:
                        direction = 2   

                elif bus == 'B54':
                    if 50 <= float(row[4]) < 250:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B57':
                    if 100 <= float(row[4]) < 270:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B62':
                    if 175 <= float(row[4]) < 350:
                        direction = 1
                    else:
                        direction = 2   

                elif bus == 'B37':
                    if 700 <= float(row[4]) < 210:
                        direction = 1
                    else:
                        direction = 2   

                elif bus == 'B41':
                    if 70 <= float(row[4]) < 200:
                        direction = 1
                    else:
                        direction = 2

                elif bus == 'B11':
                    if 60 <= float(row[4]) < 300:
                        direction = 1
                    else:
                        direction = 2   


                if len(row[7].split('_')[1].split('-')) > 3:
                    tripId = row[7].split('_')[1].split('-')[3]
                    yield row[0], bus, tripId, direction
                    
                else:
                    tripId = row[7].split('_')[1].split('-')[2]
                    yield row[0], bus, tripId, direction



In [8]:
# Wrangle Data and calculate total trip time

for month in range(1,13):
    for day in range(1,32):
        
        if month < 10 and day < 10:
            file_name = '/bus_time_2016' + '0' + str(month) + '0' + str(day)
            month_path = '0' + str(month)
        elif month >= 10 and day < 10:
            file_name = '/bus_time_2016' + str(month) + '0' + str(day)
            month_path =  str(month)
        elif month < 10 and day >= 10:
            file_name = '/bus_time_2016' + '0' + str(month)  + str(day)
            month_path =  '0' + str(month)
        elif month >= 10 and day >= 10:
            file_name = '/bus_time_2016' + str(month)  + str(day)
            month_path =  str(month)
            
            
        # Download data    
        os.system('curl -o ' + bus_data_folder + file_name + '.csv.xz ' \
                      'http://data.mytransit.nyc/bus_time/2016/2016-' + month_path + file_name + '.csv.xz')
        # Unzip
        os.system('xz -d ' + bus_data_folder + file_name + '.csv.xz')
        
        # Check if file exists
        if os.path.exists(bus_data_folder + file_name + '.csv'):
            
            # Load into spark rdd
            bus_time = sc.textFile(bus_data_folder + file_name + '.csv', 
                                           use_unicode=True).cache()

            # List of buses for analysis
            bus_list = ['B15', 'B4', 'B3', 'B8', 'B38', 'B43', 'B54', 'B57', 'B62', 'B37', \
                        'B41', 'B11']

            # Extract bus lines matching bus_list
            brooklynBuses = bus_time.mapPartitionsWithIndex(extractBuses).collect()

            # Load data into pandas dataframe
            labels = ['timestamp', 'bus_line', 'trip', 'bearing']
            df = pd.DataFrame.from_records(brooklynBuses, columns=labels)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df.sort_values(['bus_line', 'trip', 'timestamp'], inplace=True)

            # Calculate time between pings for each trip
            df['diff'] = df.groupby(['bus_line', 'trip'])['timestamp'].diff()
            df_dir = df.iloc[:,:4]
            df_red_bearing = df_dir.groupby(['bus_line', 'trip']).mean().reset_index()
            df_red_bearing.bearing = df_red_bearing.bearing.round(0)

            df_time = df[['timestamp','bus_line','trip','diff']]
            df_red_time = df_time.groupby(['bus_line', 'trip']).sum().reset_index()
            df_red = pd.merge(df_red_bearing, df_red_time, on=['bus_line','trip'], how='inner')

            df_red.columns = ['bus_line', 'trip', 'direction', 'trip_time']
            df_red['begin_time'] = df.groupby(['bus_line', 'trip']).first().reset_index()['timestamp']
            df_red['end_time'] = df.groupby(['bus_line', 'trip']).last().reset_index()['timestamp']
            df_red.to_csv(bus_data_folder + file_name + '.csv')
            
        else:
            pass
            